In [1]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum, LinExpr, QuadExpr
import gurobipy_pandas as gppd
from scipy import stats
gppd.set_interactive()

In [2]:
#set daily fish and farming luck in % (payer is in the luckiest x% of people)
fish_luck = 50
farm_luck = 50

In [3]:
m = Model('level 9')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [4]:
#define sets
crops = ['Blue Jazz', 'Cauliflower', 'Kale', 'Parsnip', 'Potato', 'Tulip', 'Green Bean', 'Strawberries']
days = [i for i in range(1,29)]

In [5]:
#define matrices (on average, you get 0.25 extra potatoes per harvest)
Crops = pd.DataFrame([[30,50,[7]], [80,175,[12]], [70,110,[6]], [20,35,[4]], 
                      [50,100,[6]], [20,30,[6]], [60, 40, [10,13,16,19,22,25]], [100, 120, [8, 12]]], 
                     columns=['buy_price', 'sell_price', 'days_to_grow'], index = crops)

In [6]:
Crops['exp_gain'] = [10, 23, 17, 8, 14, 7, 9, 18]
Crops

,buy_price,sell_price,days_to_grow,exp_gain
Blue Jazz,30,50,[7],10
Cauliflower,80,175,[12],23
Kale,70,110,[6],17
Parsnip,20,35,[4],8
Potato,50,100,[6],14
Tulip,20,30,[6],7
Green Bean,60,40,"[10, 13, 16, 19, 22, 25]",9
Strawberries,100,120,"[8, 12]",18


In [7]:
#13 out 18 hours are 4 fish available + green algue + trash, , 5 hours only 3 fish
fishodds_13 = [[0.1163, 0.2403, 0.1492, 0.1380, 0.2403, 0.1159], [0.1973, 0.2219, 0.1516, 0.1739, 0.2219, 0.0334], 
               [0.2001, 0.2226, 0.1582, 0.1787, 0.2226, 0.0177], [0.2013, 0.2225, 0.1617, 0.1810, 0.2225, 0.0110]]
fishodds_5 =  [[0.2657, 0.1642, 0.1518, 0.2657, 0.1525], [0.2701, 0.1827, 0.2109, 0.2701, 0.0668], 
               [0.2743, 0.1927, 0.2184, 0.2743, 0.0402], [0.2759, 0.1980, 0.2226, 0.2759, 0.0276]]
fp = [100, 30, 15, 75, 50, 0]
fe = [19, 8, 3, 18, 14, 3]
fishmult = [1, 1.56, 1.875, 2.25]
fisheadd = [1, 3.03, 6, 6]
farmodds = [[0.97, 0.02, 0.01], [0.69, 0.20, 0.11], [0.55, 0.28, 0.17], [0.46, 0.33, 0.21]]
farm_pay = [1, 1.25, 1.5]
farm_extra = [1,1.1,1.1,1.1]

In [8]:
mexp_val = []
mexp_sqr = []
eexp_val = []
eexp_sqr = []
fval = []
fsqr = []
for i in range(4):
    fsum = 0
    fsqsum = 0
    msqsum = 0
    msum = 0
    esqsum = 0
    esum = 0
    for k in range(3):
        fsum += farmodds[i][k]*farm_pay[k]*farm_extra[i]
        fsqsum += farmodds[i][k]*(farm_pay[k]*farm_extra[i])**2
    for j in range(6):
        if j != 5:
            msum += 5/18*fishodds_5[i][j]*fishmult[i]*fp[j+1]
            msqsum += 5/18*fishodds_5[i][j]*(fishmult[i]*fp[j+1])**2
            if fe[j+1] > 3: #green alge and trash does not get increased exp
                esum += 5/18*fishodds_5[i][j]*(fe[j+1]+fisheadd[i])
                esqsum += 5/18*fishodds_5[i][j]*(fe[j+1]+fisheadd[i])**2
            else:
                esum += 5/18*fishodds_5[i][j]*fe[j+1]
                esqsum += 5/18*fishodds_5[i][j]*fe[j+1]**2
        msum += 13/18*fishodds_13[i][j]*fishmult[i]*fp[j]
        msqsum += 13/18*fishodds_13[i][j]*(fishmult[i]*fp[j])**2
        if fe[j] > 3:
            esum += 13/18*fishodds_13[i][j]*(fe[j]+fisheadd[i])
            esqsum += 13/18*fishodds_13[i][j]*(fe[j]+fisheadd[i])**2
        else:
            esum += 13/18*fishodds_13[i][j]*fe[j]
            esqsum += 13/18*fishodds_13[i][j]*fe[j]**2
    fval.append(fsum)
    fsqr.append(fsqsum)
    eexp_val.append(esum)
    eexp_sqr.append(esqsum)
    mexp_val.append(msum)
    mexp_sqr.append(msqsum)

In [9]:
print(mexp_val)
print(eexp_val)
print(fval)
#same exp val as I calculated by hand

[41.125888888888895, 76.89157666666667, 94.04041666666666, 113.72268750000002]
[11.149555555555557, 14.223367500000002, 16.79826111111111, 16.88743333333333]
[1.01, 1.2155, 1.2705000000000002, 1.3062500000000004]


In [10]:
#calculate variances: V[X] = E[X^2] - E[X]^2
mvar = []
evar = []
favar = []
for i in range(4):
    mvar.append(mexp_sqr[i] - (mexp_val[i])**2)
    evar.append(eexp_sqr[i] - (eexp_val[i])**2)
    favar.append(fsqr[i] - (fval[i])**2)

In [11]:
#assuming normality
#the variance gets affected by the amount of crops farmed and fishes cought, 
#however it is not doable to have these huge multiplications
fish_z = stats.norm.ppf(fish_luck/100)
farm_z = stats.norm.ppf(farm_luck/100)
farm_mult = []
fish_mon = []
fish_exp = []
for i in range(4):
    farm_mult.append(round(fval[i] + farm_z*(favar[i]/20)**0.5,3))
    fish_mon.append(round(mexp_val[i] + fish_z*(mvar[i]/30)**0.5,3))
    fish_exp.append(round(eexp_val[i] + fish_z*(evar[i]/30)**0.5,3))

In [12]:
print(fish_mon)
print(fish_exp)
print(farm_mult)

[41.126, 76.892, 94.04, 113.723]
[11.15, 14.223, 16.798, 16.887]
[1.01, 1.216, 1.271, 1.306]


In [13]:
#make money matrix
Money = pd.DataFrame(index = pd.MultiIndex.from_arrays([np.repeat(days , 3), 
                                                        ['start_day', 'end_day', 'earnings']*28]).set_names(['day', 'time']))

In [14]:
Money.head(6)

Empty DataFrame
Columns: []
Index: [(1, start_day), (1, end_day), (1, earnings), (2, start_day), (2, end_day), (2, earnings)]

In [15]:
M = gppd.add_vars(m, Money, lb = 0, vtype=GRB.CONTINUOUS, name = 'money')

In [16]:
M.head(9)

day  time     
1    start_day    <gurobi.Var money[1,start_day]>
     end_day        <gurobi.Var money[1,end_day]>
     earnings      <gurobi.Var money[1,earnings]>
2    start_day    <gurobi.Var money[2,start_day]>
     end_day        <gurobi.Var money[2,end_day]>
     earnings      <gurobi.Var money[2,earnings]>
3    start_day    <gurobi.Var money[3,start_day]>
     end_day        <gurobi.Var money[3,end_day]>
     earnings      <gurobi.Var money[3,earnings]>
Name: money, dtype: object

In [17]:
#make plan variable
Plan = pd.DataFrame(index = pd.MultiIndex.from_arrays([np.repeat(days, 8),crops*28]).set_names(['day', 'crop']))

In [18]:
Plan.head(8)

Empty DataFrame
Columns: []
Index: [(1, Blue Jazz), (1, Cauliflower), (1, Kale), (1, Parsnip), (1, Potato), (1, Tulip), (1, Green Bean), (1, Strawberries)]

In [19]:
P = gppd.add_vars(m, Plan, lb = 0, vtype=GRB.INTEGER, name = 'plan')

In [20]:
P.head(16)

day  crop        
1    Blue Jazz          <gurobi.Var plan[1,Blue_Jazz]>
     Cauliflower      <gurobi.Var plan[1,Cauliflower]>
     Kale                    <gurobi.Var plan[1,Kale]>
     Parsnip              <gurobi.Var plan[1,Parsnip]>
     Potato                <gurobi.Var plan[1,Potato]>
     Tulip                  <gurobi.Var plan[1,Tulip]>
     Green Bean        <gurobi.Var plan[1,Green_Bean]>
     Strawberries    <gurobi.Var plan[1,Strawberries]>
2    Blue Jazz          <gurobi.Var plan[2,Blue_Jazz]>
     Cauliflower      <gurobi.Var plan[2,Cauliflower]>
     Kale                    <gurobi.Var plan[2,Kale]>
     Parsnip              <gurobi.Var plan[2,Parsnip]>
     Potato                <gurobi.Var plan[2,Potato]>
     Tulip                  <gurobi.Var plan[2,Tulip]>
     Green Bean        <gurobi.Var plan[2,Green_Bean]>
     Strawberries    <gurobi.Var plan[2,Strawberries]>
Name: plan, dtype: object

In [21]:
#fishing 
F = m.addVars(days, vtype = GRB.INTEGER , lb = 0, name = 'fish')

In [22]:
#binary vars for constraints
GF = m.addVars(days, vtype = GRB.BINARY, name = 'Goes_fishing')
GS = m.addVars(days, vtype = GRB.BINARY, name = 'Goes_shopping')
MM = 100000
eps = 0.01

In [23]:
ExpGain = pd.DataFrame(index = pd.MultiIndex.from_arrays(
    [np.repeat(days, 4), ['farm_exp', 'farm_tot', 'fish_exp', 'fish_tot']*28]).set_names(['day', 'action']))
ExpGain.head(8)

Empty DataFrame
Columns: []
Index: [(1, farm_exp), (1, farm_tot), (1, fish_exp), (1, fish_tot), (2, farm_exp), (2, farm_tot), (2, fish_exp), (2, fish_tot)]

In [24]:
E = gppd.add_vars(m, ExpGain, lb = 0, vtype = GRB.CONTINUOUS, name = 'exp')
E.head(12)

day  action  
1    farm_exp    <gurobi.Var exp[1,farm_exp]>
     farm_tot    <gurobi.Var exp[1,farm_tot]>
     fish_exp    <gurobi.Var exp[1,fish_exp]>
     fish_tot    <gurobi.Var exp[1,fish_tot]>
2    farm_exp    <gurobi.Var exp[2,farm_exp]>
     farm_tot    <gurobi.Var exp[2,farm_tot]>
     fish_exp    <gurobi.Var exp[2,fish_exp]>
     fish_tot    <gurobi.Var exp[2,fish_tot]>
3    farm_exp    <gurobi.Var exp[3,farm_exp]>
     farm_tot    <gurobi.Var exp[3,farm_tot]>
     fish_exp    <gurobi.Var exp[3,fish_exp]>
     fish_tot    <gurobi.Var exp[3,fish_tot]>
Name: exp, dtype: object

In [25]:
Farm_Mid = m.addVars(days, vtype = GRB.BINARY, name = 'mid_level_farming')
Farm_High = m.addVars(days, vtype= GRB.BINARY, name = 'high_level_farming')
Farm_Max = m.addVars(days, vtype = GRB.BINARY, name = 'max_level_farming')

In [26]:
Fish_Mid = m.addVars(days, vtype = GRB.BINARY, name = 'mid_level_fishing')
Fish_High = m.addVars(days, vtype = GRB.BINARY, name = 'high_level_fishing')
Fish_Max = m.addVars(days, vtype = GRB.BINARY, name = 'max_level_fishing')

In [27]:
m.setObjective(M[28, 'end_day'] + M[28, 'earnings'], sense= GRB.MAXIMIZE )

In [28]:
#c2 : Money end day = money star day - prices seeds you bought
#c3 : Earnings days to grow after you bought & planted seeds = sell price of those seeds + fish caught that day
#c8 : total energy usage in a day (watering of all standing crops, yet to be harvested; 
#     land plouging; fishing) <= 270 (max energy)
#c11 : all time constrained. 6 * 18 blocks of 10 min in game time. 
#c12 : sum P[j][i] for all i in crops <= alot * GS [j]
#c13 : F[j] <= alot * GF[j] 
#c16 : farming exp earned upon harvest (for multiple harvests multiple times exp)
#c17 : fishing exp earned upon cast
#c18 : alot * Farm mid/high/max
#c19 : alot * Fish mid/high/max


c2 = pd.Series(index = days, dtype = object)
c3 = pd.Series(index = days, dtype = object)
c3_1 = pd.Series(index = days, dtype = object)
c3_2 = pd.Series(index = days, dtype = object)
c3_3 = pd.Series(index = days, dtype = object)
c3_4 = pd.Series(index = days, dtype = object)
c3_5 = pd.Series(index = days, dtype = object)
c3_6 = pd.Series(index = days, dtype = object)
c3_7 = pd.Series(index = days, dtype = object)
c3_8 = pd.Series(index = days, dtype = object)
c3_9 = pd.Series(index = days, dtype = object)
c3_10 = pd.Series(index = days, dtype = object)
c3_11 = pd.Series(index = days, dtype = object)
c3_12 = pd.Series(index = days, dtype = object)
c3_13 = pd.Series(index = days, dtype = object)
c3_14 = pd.Series(index = days, dtype = object)
c3_15 = pd.Series(index = days, dtype = object)
c3_16 = pd.Series(index = days, dtype = object)
c3_17 = pd.Series(index = days, dtype = object)
c3_18 = pd.Series(index = days, dtype = object)
c3_19 = pd.Series(index = days, dtype = object)
c3_20 = pd.Series(index = days, dtype = object)
c3_21 = pd.Series(index = days, dtype = object)
c3_22 = pd.Series(index = days, dtype = object)
c3_23 = pd.Series(index = days, dtype = object)
c3_24 = pd.Series(index = days, dtype = object)
c3_25 = pd.Series(index = days, dtype = object)
c3_26 = pd.Series(index = days, dtype = object)
c3_27 = pd.Series(index = days, dtype = object)
c3_28 = pd.Series(index = days, dtype = object)
c3_29 = pd.Series(index = days, dtype = object)
c3_30 = pd.Series(index = days, dtype = object)
c3_31 = pd.Series(index = days, dtype = object)
c8 = pd.Series(index = days, dtype = object)
c8_1 = pd.Series(index = days, dtype = object)
c8_2 = pd.Series(index = days, dtype = object)
c8_3 = pd.Series(index = days, dtype = object)
c8_4 = pd.Series(index = days, dtype=object)
c8_5 = pd.Series(index = days, dtype=object)
c8_6 = pd.Series(index = days, dtype=object)
c8_7 = pd.Series(index = days, dtype=object)
c8_8 = pd.Series(index = days, dtype=object)
c8_9 = pd.Series(index = days, dtype=object)
c8_10 = pd.Series(index = days, dtype=object)
c8_11 = pd.Series(index = days, dtype=object)
c8_12 = pd.Series(index = days, dtype=object)
c8_13 = pd.Series(index = days, dtype=object)
c8_14 = pd.Series(index = days, dtype=object)
c8_15 = pd.Series(index = days, dtype=object)
c11 = pd.Series(index = days, dtype = object)
c12 = pd.Series(index = days, dtype = object)
c13 = pd.Series(index = days, dtype = object)
c16 = pd.Series(index = days, dtype = object)
c17 = pd.Series(index = days, dtype = object)
c17_1 = pd.Series(index = days, dtype = object)
c17_2 = pd.Series(index = days, dtype = object)
c17_3 = pd.Series(index = days, dtype = object)
c17_4 = pd.Series(index = days, dtype = object)
c17_5 = pd.Series(index = days, dtype = object)
c17_6 = pd.Series(index = days, dtype = object)
c17_7 = pd.Series(index = days, dtype = object)
c18 = pd.Series(index = days, dtype = object)
c18_1 = pd.Series(index = days, dtype = object)
c18_2 = pd.Series(index = days, dtype = object)
c18_3 = pd.Series(index = days, dtype = object)
c18_4 = pd.Series(index = days, dtype = object)
c18_5 = pd.Series(index = days, dtype = object)
c19 = pd.Series(index = days, dtype = object)
c19_1 = pd.Series(index = days, dtype = object)
c19_2 = pd.Series(index = days, dtype = object)
c19_3 = pd.Series(index = days, dtype = object)
c19_4 = pd.Series(index = days, dtype = object)
c19_5 = pd.Series(index = days, dtype = object)

for j in range(28, 0, -1):
    tot_mon = LinExpr(0)
    tot_mon_1 = LinExpr(0)
    tot_mon_2 = LinExpr(0)
    tot_mon_3 = LinExpr(0)
    tot_ene = LinExpr(0)
    tot_ene_1 = LinExpr(0)
    tot_ene_2 = LinExpr(0)
    tot_ene_3 = LinExpr(0)
    tot_time = LinExpr(0)
    tot_exp = LinExpr(0)
    for i in crops:
        gen = [k for k in Crops.loc[i,'days_to_grow'] if j-k > 0]
        
        # days watering
        if len(gen) > 0:
            for k in range(1, max(gen)):
                tot_ene += 2 * P[j-k][i] 
                tot_ene_1 += 1.5*P[j-k][i]
                tot_ene_2 += 1.2*P[j-k][i]
                tot_ene_3 += P[j-k][i]
                tot_time += 0.125 * P[j-k][i]
        
        #days planting
        tot_ene += 4*P[j][i] 
        tot_ene_1 += 3*P[j][i]
        tot_ene_2 += 2.4*P[j][i]
        tot_ene_3 += 2*P[j][i]
        tot_time += 0.225 * P[j][i]
        
        #harvest days
        tot_time += quicksum(0.05 * P[j-k][i] for k in gen)
        tot_mon += quicksum(farm_mult[0]*P[j-k][i]*Crops.loc[i, 'sell_price'] for k in gen)
        tot_mon_1 += quicksum(farm_mult[1]*P[j - k][i] * Crops.loc[i, 'sell_price'] for k in gen)
        tot_mon_2 += quicksum(farm_mult[2]*P[j - k][i] * Crops.loc[i, 'sell_price'] for k in gen)
        tot_mon_3 += quicksum(farm_mult[3]*P[j - k][i] * Crops.loc[i, 'sell_price'] for k in gen)
        tot_exp += quicksum(P[j - k][i] * Crops.loc[i, 'exp_gain'] for k in gen)
    
    c2.loc[j] = m.addConstr(M[j]['end_day'] == M[j]['start_day'] - quicksum(Crops['buy_price'][i]*P[j][i] for i in crops))
    
    c3.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon + fish_mon[0]*F[j] + MM*Farm_Mid[j] + MM*Fish_Mid[j])
    c3_1.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon + fish_mon[0]*F[j] - MM*Farm_Mid[j] - MM*Fish_Mid[j])
    c3_2.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_1 + fish_mon[0]*F[j] + MM*(1-Farm_Mid[j]+Farm_High[j]) + MM*Fish_Mid[j])
    c3_3.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_1 + fish_mon[0]*F[j] - MM*(1-Farm_Mid[j]+Farm_High[j]) - MM*Fish_Mid[j])
    c3_4.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_2 + fish_mon[0]*F[j] + MM*(1-Farm_High[j]+Farm_Max[j]) + MM*Fish_Mid[j])
    c3_5.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_2 + fish_mon[0]*F[j] - MM*(1-Farm_High[j]+Farm_Max[j]) - MM*Fish_Mid[j])
    c3_6.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_3 + fish_mon[0]*F[j] + MM*(1-Farm_Max[j]) + MM*Fish_Mid[j])
    c3_7.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_3 + fish_mon[0]*F[j] - MM*(1-Farm_Max[j]) - MM*Fish_Mid[j])
    c3_8.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon + fish_mon[1]*F[j] + MM*Farm_Mid[j] + MM*(1-Fish_Mid[j] + Fish_High[j]))
    c3_9.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon + fish_mon[1]*F[j] - MM*Farm_Mid[j] - MM*(1-Fish_Mid[j] + Fish_High[j]))
    c3_10.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_1 + fish_mon[1]*F[j] + MM*(1-Farm_Mid[j]+Farm_High[j]) + MM*(1-Fish_Mid[j] + Fish_High[j]))
    c3_11.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_1 + fish_mon[1]*F[j] - MM*(1-Farm_Mid[j]+Farm_High[j]) - MM*(1-Fish_Mid[j] + Fish_High[j]))
    c3_12.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_2 + fish_mon[1]*F[j] + MM*(1-Farm_High[j]+Farm_Max[j]) + MM*(1-Fish_Mid[j] + Fish_High[j]))
    c3_13.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_2 + fish_mon[1]*F[j] - MM*(1-Farm_High[j]+Farm_Max[j]) - MM*(1-Fish_Mid[j] + Fish_High[j]))
    c3_14.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_3 + fish_mon[1]*F[j] + MM*(1-Farm_Max[j]) + MM*(1-Fish_Mid[j] + Fish_High[j]))
    c3_15.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_3 + fish_mon[1]*F[j] - MM*(1-Farm_Max[j]) - MM*(1-Fish_Mid[j] + Fish_High[j]))
    c3_16.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon + fish_mon[2]*F[j] + MM*Farm_Mid[j] + MM*(1-Fish_High[j] + Fish_Max[j]))
    c3_17.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon + fish_mon[2]*F[j] - MM*Farm_Mid[j] - MM*(1-Fish_High[j] + Fish_Max[j]))
    c3_18.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_1 + fish_mon[2]*F[j] + MM*(1-Farm_Mid[j]+Farm_High[j]) + MM*(1-Fish_High[j] + Fish_Max[j]))
    c3_19.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_1 + fish_mon[2]*F[j] - MM*(1-Farm_Mid[j]+Farm_High[j]) - MM*(1-Fish_High[j] + Fish_Max[j]))
    c3_20.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_2 + fish_mon[2]*F[j] + MM*(1-Farm_High[j]+Farm_Max[j]) + MM*(1-Fish_High[j] + Fish_Max[j]))
    c3_21.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_2 + fish_mon[2]*F[j] - MM*(1-Farm_High[j]+Farm_Max[j]) - MM*(1-Fish_High[j] + Fish_Max[j]))
    c3_22.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_3 + fish_mon[2]*F[j] + MM*(1-Farm_Max[j]) + MM*(1-Fish_High[j] + Fish_Max[j]))
    c3_23.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_3 + fish_mon[2]*F[j] - MM*(1-Farm_Max[j]) - MM*(1-Fish_High[j] + Fish_Max[j]))
    c3_24.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon + fish_mon[3]*F[j] + MM*Farm_Mid[j] + MM*(1-Fish_Max[j]))
    c3_25.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon + fish_mon[3]*F[j] - MM*Farm_Mid[j] - MM*(1-Fish_Max[j]))
    c3_26.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_1 + fish_mon[3]*F[j] + MM*(1-Farm_Mid[j]+Farm_High[j]) + MM*(1-Fish_Max[j]))
    c3_27.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_1 + fish_mon[3]*F[j] - MM*(1-Farm_Mid[j]+Farm_High[j]) - MM*(1-Fish_Max[j]))
    c3_28.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_2 + fish_mon[3]*F[j] + MM*(1-Farm_High[j]+Farm_Max[j]) + MM*(1-Fish_Max[j]))
    c3_29.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_2 + fish_mon[3]*F[j] - MM*(1-Farm_High[j]+Farm_Max[j]) - MM*(1-Fish_Max[j]))
    c3_30.loc[j] = m.addConstr(M[j]['earnings'] <= tot_mon_3 + fish_mon[3]*F[j] + MM*(1-Farm_Max[j]) + MM*(1-Fish_Max[j]))
    c3_31.loc[j] = m.addConstr(M[j]['earnings'] >= tot_mon_3 + fish_mon[3]*F[j] - MM*(1-Farm_Max[j]) - MM*(1-Fish_Max[j]))
    
    c8.loc[j] = m.addConstr(tot_ene + 8*F[j] <= 270 + MM*Farm_Mid[j] +MM*Fish_Mid[j]) 
    c8_1.loc[j] = m.addConstr(tot_ene_1 + 8*F[j] <= 270 + MM*Farm_High[j] + MM*Fish_Mid[j])
    c8_2.loc[j] = m.addConstr(tot_ene_2 + 8*F[j] <= 270 + MM*Farm_Max[j] + MM*Fish_Mid[j])
    c8_3.loc[j] = m.addConstr(tot_ene_3 + 8*F[j] <= 270 + MM*Fish_Mid[j]) 
    c8_4.loc[j] = m.addConstr(tot_ene + 7.5*F[j] <= 270 + MM*Farm_Mid[j] + MM*Fish_High[j])
    c8_5.loc[j] = m.addConstr(tot_ene_1 + 7.5*F[j] <= 270 + MM*Farm_High[j] + MM*Fish_High[j])
    c8_6.loc[j] = m.addConstr(tot_ene_2 + 7.5*F[j] <= 270 + MM*Farm_Max[j] + MM*Fish_High[j])
    c8_7.loc[j] = m.addConstr(tot_ene_3 + 7.5*F[j] <= 270 + MM*Fish_High[j])
    c8_8.loc[j] = m.addConstr(tot_ene + 7.2*F[j] <= 270 + MM*Farm_Mid[j] + MM*Fish_Max[j])
    c8_9.loc[j] = m.addConstr(tot_ene_1 + 7.2*F[j] <= 270 + MM*Farm_High[j] + MM*Fish_Max[j])
    c8_10.loc[j] = m.addConstr(tot_ene_2 + 7.2*F[j] <= 270 + MM*Farm_Max[j] + MM*Fish_Max[j])
    c8_11.loc[j] = m.addConstr(tot_ene_3 + 7.2*F[j] <= 270 + MM*Fish_Max[j])
    c8_12.loc[j] = m.addConstr(tot_ene + 7*F[j] <= 270 + MM*Farm_Mid[j])
    c8_13.loc[j] = m.addConstr(tot_ene_1 + 7*F[j] <= 270 + MM*Farm_High[j])
    c8_14.loc[j] = m.addConstr(tot_ene_2 + 7*F[j] <= 270 + MM*Farm_Max[j])
    c8_15.loc[j] = m.addConstr(tot_ene_3 + 7*F[j] <= 270)
    
    if j == 2:
        c11.loc[j] = m.addConstr(tot_time + 3*F[j] + 16*GF[j] + 6*GS[j] <= 105, name = 'time_' + str(j))
    else:
        c11.loc[j] = m.addConstr(tot_time + 3*F[j] + 10*GF[j] + 6*GS[j] <= 105, name = 'time_' + str(j))
    
    c12.loc[j] = m.addConstr(quicksum(P[j][i] for i in crops) <= MM*GS[j])
    c13.loc[j] = m.addConstr(F[j] <= MM*GF[j])
    c16.loc[j] = m.addConstr(E[j]['farm_exp'] == tot_exp, name = 'fa_exp_' + str(j))
    
    c17.loc[j] = m.addConstr(E[j]['fish_exp'] <= fish_exp[0]*F[j] + MM*Fish_Mid[j])
    c17_1.loc[j] = m.addConstr(E[j]['fish_exp'] >= fish_exp[0]*F[j] - MM*Fish_Mid[j])
    c17_2.loc[j] = m.addConstr(E[j]['fish_exp'] <= fish_exp[1]*F[j] + MM*(1-Fish_Mid[j] + Fish_High[j]))
    c17_3.loc[j] = m.addConstr(E[j]['fish_exp'] >= fish_exp[1]*F[j] - MM*(1-Fish_Mid[j] + Fish_High[j]))
    c17_4.loc[j] = m.addConstr(E[j]['fish_exp'] <= fish_exp[2]*F[j] + MM*(1-Fish_High[j] + Fish_Max[j]))
    c17_5.loc[j] = m.addConstr(E[j]['fish_exp'] >= fish_exp[2]*F[j] - MM*(1-Fish_High[j] + Fish_Max[j]))
    c17_6.loc[j] = m.addConstr(E[j]['fish_exp'] <= fish_exp[3]*F[j] + MM*(1-Fish_Max[j]))
    c17_7.loc[j] = m.addConstr(E[j]['fish_exp'] >= fish_exp[3]*F[j] - MM*(1-Fish_Max[j]))
    
    c18.loc[j] = m.addConstr(E[j]['farm_tot'] >= 2150 - MM*(1-Farm_Mid[j]))
    c18_1.loc[j] = m.addConstr(E[j]['farm_tot'] <= 2150 - eps + MM*Farm_Mid[j])
    c18_2.loc[j] = m.addConstr(E[j]['farm_tot'] >= 6900 - MM*(1-Farm_High[j]))
    c18_3.loc[j] = m.addConstr(E[j]['farm_tot'] <= 6900 - eps + MM*Farm_High[j])
    c18_4.loc[j] = m.addConstr(E[j]['farm_tot'] >= 15000 - MM*(1-Farm_Max[j]))
    c18_5.loc[j] = m.addConstr(E[j]['farm_tot'] <= 15000 - eps + MM*Farm_Max[j])
    
    c19.loc[j] = m.addConstr(E[j]['fish_tot'] >= 2150 - MM*(1-Fish_Mid[j]))
    c19_1.loc[j] = m.addConstr(E[j]['fish_tot'] <= 2150 - eps + MM*Fish_Mid[j])
    c19_2.loc[j] = m.addConstr(E[j]['fish_tot'] >= 6900 - MM*(1-Fish_High[j]))
    c19_3.loc[j] = m.addConstr(E[j]['fish_tot'] <= 6900 - eps + MM*Fish_High[j])
    c19_4.loc[j] = m.addConstr(E[j]['fish_tot'] >= 15000 - MM*(1-Fish_Max[j]))
    c19_5.loc[j] = m.addConstr(E[j]['fish_tot'] <= 15000 - eps + MM*Fish_Max[j])
    
    m.update()

m.update()

In [29]:
#c4 : starting money the next day = earnings + ending money from the previous day

c4 = pd.Series(index= [k for k in range (1,28)], dtype = object)
for i in range(1, 28):
    c4.loc[i] = m.addConstr(M[i+1]['start_day'] == M[i]['end_day'] + M[i]['earnings'], name = 'c4_'+str(i))
m.update()

In [30]:
#c5 : start with 15 parsnip seeds & 500 money

c5 = m.addConstr(M[1]['start_day'] == 800)
c5_1 = m.addConstr(P[1]['Parsnip'] >= 15)
m.update()

In [31]:
#c6 : Can't buy other things then strawberries on day 13

c6 = m.addConstr(quicksum(P[13][i] for i in [x for x in crops if x != 'Strawberries']) == 0, name='c6')

In [32]:
#c7 : Can't buy strawberries on any other day then day 13

gen = (i for i in days if i != 13)
c7 = m.addConstr(quicksum(P[i]['Strawberries'] for i in gen)== 0)
m.update()

In [33]:
#c9 : can't buy seeds on wednesdays (24th is also flowerdance day)

c9 = m.addConstr(quicksum(P[i][j] for i in [3,10,17,24] for j in crops) == 0)
m.update()

In [34]:
#c10 : fishing starts on day 2, can't fish on the 13th
c10 = m.addConstr(F[1] + F[13] == 0)
m.update()

In [35]:
c14 = pd.Series([k for k in range(1,28)], dtype=object)
c14_1 = pd.Series([k for k in range(1,28)], dtype = object)
for i in range(1,28):
    c14.loc[i] = m.addConstr(E[i+1]['farm_tot'] == E[i]['farm_tot'] + E[i]['farm_exp'], name = 'c14_' + str(i))
    c14_1.loc[i] = m.addConstr(E[i+1]['fish_tot'] == E[i]['fish_tot'] + E[i]['fish_exp'], name = 'c14_1_' + str(i))
m.update()

In [36]:
c15 = m.addConstr(E[1]['farm_tot'] + E[1]['fish_tot'] == 0, name = 'c15')
m.update()

In [37]:
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2132 rows, 672 columns and 39987 nonzeros
Model fingerprint: 0x7c211ec6
Variable types: 196 continuous, 476 integer (224 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+05]
Presolve removed 1185 rows and 303 columns
Presolve time: 0.12s
Presolved: 947 rows, 369 columns, 9629 nonzeros
Variable types: 69 continuous, 300 integer (115 binary)
Found heuristic solution: objective 11712.662000

Root relaxation: objective 2.314981e+06, 477 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

In [38]:
p = pd.DataFrame(np.array([int(p.X) for p in P]).reshape(len(days), len(crops)), index = days, columns=crops)
p['Fish'] = [int(F[i].X) for i in days]
p

,Blue Jazz,Cauliflower,Kale,Parsnip,Potato,Tulip,Green Bean,Strawberries,Fish
1,0,0,0,15,0,0,8,0,0
2,0,0,0,0,0,0,0,0,29
3,0,0,0,0,0,0,0,0,31
4,0,30,0,0,1,0,0,0,18
5,0,7,0,0,4,0,0,0,28
6,0,8,0,0,0,0,0,0,29
7,0,0,0,0,0,0,0,0,31
8,0,0,5,0,6,0,0,0,27
9,0,0,0,0,0,0,0,0,31
10,0,0,0,0,0,0,0,0,31


In [39]:
mo = pd.DataFrame(np.array([int(m.X) for m in M]).reshape(len(days), 3), index = days, 
                  columns=['start_day', 'end_day', 'earnings'])
mo.loc[29] = [int(m.getAttr(attrname='ObjVal')), 'NVT' , 'NVT']
mo

,start_day,end_day,earnings
1,800,20,0
2,20,20,1192
3,1212,1212,1274
4,2487,37,740
5,777,17,1681
6,1699,1059,1192
7,2252,2252,1274
8,3527,2877,1110
9,3987,3987,2383
10,6371,6371,2484


In [40]:
e = pd.DataFrame(np.array([int(e.X) for e in E]).reshape(len(days), 4), 
                 index = days, columns = ['farm exp earned', 'tot farm exp','fish exp earned', 'tot fish exp'])
e['level farming'] = [['low', 'mid', 'high', 'max'][int(Farm_Mid[i].X + Farm_High[i].X + Farm_Max[i].X)] for i in days]
e['level fishing'] = [['low', 'mid', 'high', 'max'][int(Fish_Mid[i].X + Fish_High[i].X + Fish_Max[i].X)] for i in days]
e

,farm exp earned,tot farm exp,fish exp earned,tot fish exp,level farming,level fishing
1,0,0,0,0,low,low
2,0,0,323,0,low,low
3,0,0,345,323,low,low
4,0,0,200,669,low,low
5,120,0,312,869,low,low
6,0,120,323,1181,low,low
7,0,120,345,1505,low,low
8,0,120,301,1850,low,low
9,0,120,440,2151,low,mid
10,14,120,440,2592,low,mid
